# 中央競馬ポジションバイアス

In [4]:
import time
import pandas as pd
from urllib.request import urlopen
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
import re
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

In [5]:
race_name_list = ['2回新潟3日','3回中京3日','1回札幌3日']
race_num_list = ['1レース','2レース','3レース','4レース','5レース','6レース','7レース','8レース','9レース','10レース','11レース','12レース']

# JRA情報取得レースリスト作成

In [6]:
# 宣言
# 宣言
#driver = webdriver.Chrome(ChromeDriverManager().install())
#driver = webdriver.Chrome(executable_path=".\chromedriver")
#driver = webdriver.Chrome()

# ChromeDriverの自動管理を使用
options = Options()
# SSLエラー対策
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
options.add_argument('--allow-insecure-localhost')

# ウィンドウサイズの設定
options.add_argument('--window-size=200,200')  # 幅400px、高さ400px

# 安定性のための設定
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# FFmpegエラーメッセージを抑制したい場合は以下を追加
options.add_argument('--disable-logging')
options.add_experimental_option('excludeSwitches', ['enable-logging'])

# WebGLエラー対策
options.add_argument('--enable-unsafe-swiftshader')  # SwiftShaderを有効化[1]
options.add_argument('--ignore-gpu-blocklist')  # GPUブロックリストを無視

# WebDriverManagerは不要になり、直接ChromeDriverを初期化できる
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)  # タイムアウト10秒

df = pd.DataFrame()

url = 'https://www.jra.go.jp/'
driver.get(url)
time.sleep(1)

# 結果ページへ移動
driver.find_element(By.CSS_SELECTOR,'#quick_menu > div > ul > li:nth-child(4) > a > div:nth-child(1) > img').click()
time.sleep(1)

# 競馬場選択
for course in race_name_list:
    driver.find_element(By.LINK_TEXT,course).click()
    time.sleep(1)
    # レース番号単位でページ遷移
    for race_num in race_num_list:
        imgs = driver.find_elements(By.TAG_NAME,'img')
        for img in imgs:
            alttxt = img.get_attribute('alt')
            if alttxt == race_num:
                nlabel = course + race_num
                #print(course + race_num)
                img.click()
                time.sleep(1)
                #障害レースはカウント対象外
                if '障害' in driver.find_element(By.CLASS_NAME,'cell.category').text:
                    time.sleep(2)
                    break
                ctype = driver.find_element(By.CLASS_NAME,'cell.course').find_element(By.CLASS_NAME,'detail').text
                #1000直はカウント対象外
                if ctype.find('直') != -1:
                    time.sleep(2)
                    break
                elif '芝' in ctype:
                    ctype = '芝'
                else:
                    ctype = 'ダ'
                # 実データ取得
                tds = driver.find_elements(By.TAG_NAME,'td')
                row = []
                for td in tds:
                    if td.get_attribute('class') in ['place','waku','horse','corner','pop']:
                        if td.get_attribute('class') == 'waku':
                            row.append(re.findall(r'\d+', td.find_element(By.TAG_NAME,('img')).get_attribute('alt')))
                        elif td.get_attribute('class') == 'pop':
                            row.append(td.text)
                            row.append(ctype)
                            #df = df.append(pd.Series(row, name=nlabel))
                            df2 = pd.DataFrame([row])
                            df = pd.concat([df, df2.rename(index={idx: nlabel for idx in df2.index})])
                            row.clear()
                        else:
                            row.append(td.text)
                break

driver.close()


In [7]:
#通過順を取得し、脚質分け
passtype = []
for passages in df[3]:
    passage = passages.split(' ')
    #文字列を数値に変換
    try:
        l_passage = [int(s) for s in passage]
        if max(l_passage) <= 4:
            passtype.append('A')
        elif min(l_passage) >= 5:
            passtype.append('B')
        else:
            passtype.append('C')
    except:
        passtype.append('C')
        
#タ脚質分けした値をdataframeに追加
df['脚質'] = passtype
#各列の名前を追加
df.columns = ['着順','枠番','馬名','通過順','人気','種別','脚質']

In [8]:
df.to_pickle('./pbias.pickle')

In [9]:
df = pd.read_pickle('./pbias.pickle')

In [10]:
for course in race_name_list:
    #枠番のポジションバイアス無効判定
    s_cnt = 0
    d_cnt = 0
    for race in set(df.index.values):
        if course in race:
            df_check = df[race:race]
            #出走頭数が14頭以上
            if len( df_check[df_check['着順'].isin(['14'])]) > 0:
                for ctype in df_check['種別']:
                    if ctype == 'ダ':
                        d_cnt = d_cnt + 1
                    else:
                        s_cnt = s_cnt + 1
                    break

    s_kyaku = [[0,0,0],[0,0,0],[0,0,0]]
    s_waku = [[0,0],[0,0],[0,0]]
    d_kyaku = [[0,0,0],[0,0,0],[0,0,0]]
    d_waku = [[0,0],[0,0],[0,0]]
    for index, data in df.iterrows():
        #対象コース名に含まれるとき
        if course in index:
            if data['種別'] == '芝':
                kyaku = s_kyaku
                waku = s_waku
            else:
                kyaku = d_kyaku
                waku = d_waku
            
            if data['着順'] in ['1','2','3']:
                #脚質　基本
                if data['脚質'] == 'A':
                    kyaku[0][0] = kyaku[0][0] + 1
                elif data['脚質'] == 'B':
                    kyaku[0][1] = kyaku[0][1] + 1
                else:
                    kyaku[0][2] = kyaku[0][2] + 1
                
                #脚質　穴
                if int(data['人気']) > 3:
                    if data['脚質'] == 'A':
                        kyaku[2][0] = kyaku[2][0] + 1
                    elif data['脚質'] == 'B':
                        kyaku[2][1] = kyaku[2][1] + 1
                    else:
                        kyaku[2][2] = kyaku[2][2] + 1

                if len(df[index:index]) > 13:
                    #内外　基本
                    if int(data['枠番'][0][0]) < 5:
                        waku[0][0] = waku[0][0] + 1
                        if int(data['人気']) > 3:
                            waku[2][0] = waku[2][0] + 1
                    #内外　穴
                    else:
                        waku[0][1] = waku[0][1] + 1
                        if int(data['人気']) > 3:
                            waku[2][1] = waku[2][1] + 1
                
            if data['着順'] in ['1','2']:
                #脚質　連対
                if data['脚質'] == 'A':
                    kyaku[1][0] = kyaku[1][0] + 1
                elif data['脚質'] == 'B':
                    kyaku[1][1] = kyaku[1][1] + 1
                else:
                    kyaku[1][2] = kyaku[1][2] + 1
                    
                if len(df[index:index]) > 13:
                    #内外　連対
                    if int(data['枠番'][0][0]) < 5:
                        waku[1][0] = waku[1][0] + 1
                    else:
                        waku[1][1] = waku[1][1] + 1

    print('◆' + course)
    print('　●脚質 芝')
    strlist = ['複勝圏','連対圏','穴好走']
    i = 0
    for ky in s_kyaku:
        print('　　'+strlist[i], end=' : ')
        bplist = [ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)]
        if bplist[0] >= 0.7:
            print('★前{0:.2%}/後{1:.2%}/他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        elif bplist[1] >= 0.7:
            print('前{0:.2%}/★後{1:.2%}/他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        elif bplist[2] >= 0.7:
            print('前{0:.2%}/後{1:.2%}/★他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        else:
            print('前{0:.2%}/後{1:.2%}/他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        
        if strlist[i] == '穴好走':
            if sum(ky) <= 5:
                print('※サンプル不足※')
            else:
                print()
        else:
            print()
            
        i = i+1
    print('　●枠番 芝')
    if s_cnt > 2:
        i = 0
        for wa in s_waku:
            print('　　'+strlist[i], end=' : ')
            bplist = [wa[0]/sum(wa),wa[1]/sum(wa)]
            if bplist[0] >= 0.7:
                print('★内{0:.2%}/外{1:.2%}'.format(wa[0]/sum(wa),wa[1]/sum(wa)), end=' (')
                print('内{0:2d}/外{1:2d}/計{2:2d})'.format(wa[0],wa[1],sum(wa)), end=' ')
            elif bplist[1] >= 0.7:
                print('内{0:.2%}/★外{1:.2%}'.format(wa[0]/sum(wa),wa[1]/sum(wa)), end=' (')
                print('内{0:2d}/外{1:2d}/計{2:2d})'.format(wa[0],wa[1],sum(wa)), end=' ')
            else:
                print('内{0:.2%}/外{1:.2%}'.format(wa[0]/sum(wa),wa[1]/sum(wa)), end=' (')
                print('内{0:2d}/外{1:2d}/計{2:2d})'.format(wa[0],wa[1],sum(wa)), end=' ')
                
            if strlist[i] == '穴好走':
                if sum(wa) <= 5:
                    print('※サンプル不足※')
                else:
                    print()
            else:
                print()

            i = i+1
        print()
    else:
        print('　　※サンプル不足のため無効※')

    print('　●脚質 ダート')
    i = 0
    for ky in d_kyaku:
        print('　　'+strlist[i], end=' : ')
        bplist = [ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)]
        if bplist[0] >= 0.7:
            print('★前{0:.2%}/後{1:.2%}/他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        elif bplist[1] >= 0.7:
            print('前{0:.2%}/★後{1:.2%}/他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        elif bplist[2] >= 0.7:
            print('前{0:.2%}/後{1:.2%}/★他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        else:
            print('前{0:.2%}/後{1:.2%}/他{2:.2%}'.format(ky[0]/sum(ky),ky[1]/sum(ky),ky[2]/sum(ky)), end=' (')
            print('前{0:2d}/後{1:2d}/他{2:2d}/計{3:2d}) '.format(ky[0],ky[1],ky[2],sum(ky)), end=' ')
        
        if strlist[i] == '穴好走':
            if sum(ky) <= 5:
                print('※サンプル不足※')
            else:
                print()
        else:
            print()
            
        i = i+1
    print('　●枠番 ダート')
    if d_cnt > 2:
        i = 0
        for wa in d_waku:
            print('　　'+strlist[i], end=' : ')
            bplist = [wa[0]/sum(wa),wa[1]/sum(wa)]
            if bplist[0] >= 0.7:
                print('★内{0:.2%}/外{1:.2%}'.format(wa[0]/sum(wa),wa[1]/sum(wa)), end=' (')
                print('内{0:2d}/外{1:2d}/計{2:2d})'.format(wa[0],wa[1],sum(wa)), end=' ')
            elif bplist[1] >= 0.7:
                print('内{0:.2%}/★外{1:.2%}'.format(wa[0]/sum(wa),wa[1]/sum(wa)), end=' (')
                print('内{0:2d}/外{1:2d}/計{2:2d})'.format(wa[0],wa[1],sum(wa)), end=' ')
            else:
                print('内{0:.2%}/外{1:.2%}'.format(wa[0]/sum(wa),wa[1]/sum(wa)), end=' (')
                print('内{0:2d}/外{1:2d}/計{2:2d})'.format(wa[0],wa[1],sum(wa)), end=' ')
                
            if strlist[i] == '穴好走':
                if sum(wa) <= 5:
                    print('※サンプル不足※')
                else:
                    print()
            else:
                print()
            i = i+1
    else:
        print('　　※サンプル不足のため無効※')
        
    print()


◆2回新潟3日
　●脚質 芝
　　複勝圏 : 前58.33%/後41.67%/他0.00% (前 7/後 5/他 0/計12)  
　　連対圏 : 前37.50%/後62.50%/他0.00% (前 3/後 5/他 0/計 8)  
　　穴好走 : 前60.00%/後40.00%/他0.00% (前 3/後 2/他 0/計 5)  ※サンプル不足※
　●枠番 芝
　　※サンプル不足のため無効※
　●脚質 ダート
　　複勝圏 : 前46.67%/後20.00%/他33.33% (前 7/後 3/他 5/計15)  
　　連対圏 : 前60.00%/後10.00%/他30.00% (前 6/後 1/他 3/計10)  
　　穴好走 : 前62.50%/後12.50%/他25.00% (前 5/後 1/他 2/計 8)  
　●枠番 ダート
　　複勝圏 : 内22.22%/★外77.78% (内 2/外 7/計 9) 
　　連対圏 : 内16.67%/★外83.33% (内 1/外 5/計 6) 
　　穴好走 : 内16.67%/★外83.33% (内 1/外 5/計 6) 

◆3回中京3日
　●脚質 芝
　　複勝圏 : 前33.33%/後44.44%/他22.22% (前 6/後 8/他 4/計18)  
　　連対圏 : 前33.33%/後41.67%/他25.00% (前 4/後 5/他 3/計12)  
　　穴好走 : 前14.29%/★後71.43%/他14.29% (前 1/後 5/他 1/計 7)  
　●枠番 芝
　　※サンプル不足のため無効※
　●脚質 ダート
　　複勝圏 : 前33.33%/後44.44%/他22.22% (前 6/後 8/他 4/計18)  
　　連対圏 : 前33.33%/後41.67%/他25.00% (前 4/後 5/他 3/計12)  
　　穴好走 : 前28.57%/後42.86%/他28.57% (前 2/後 3/他 2/計 7)  
　●枠番 ダート
　　複勝圏 : 内33.33%/外66.67% (内 3/外 6/計 9) 
　　連対圏 : 内33.33%/外66.67% (内 2/外 4/計 6) 
　　穴好走 : 内33.33%/外66.67% (内 1/外 2/計 3) ※サンプル不足※

◆1回札幌3日
　●脚

In [5]:
JRA_course_dict = {'01':'札幌','02':'函館','03':'福島','04':'新潟','05':'東京','06':'中山','07':'中京','08':'京都','09':'阪神','10':'小倉','B1':'レパ'}
JRA_R_course_dict = {'札幌':'1','函館':'2','福島':'3','新潟':'4','東京':'5','中山':'6','中京':'7','京都':'8','阪神':'9','小倉':'10','レパ':'B1'}

date = 20250201

In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import re

# Ensure options is defined
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
options.add_argument('--allow-insecure-localhost')
options.add_argument('--window-size=200,200')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-logging')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--enable-unsafe-swiftshader')
options.add_argument('--ignore-gpu-blocklist')

df_total = pd.DataFrame()
race_id_list = []
url = 'https://race.netkeiba.com/top/race_list_sub.html?kaisai_date=' + str(date)
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)  # タイムアウト10秒
driver.get(url)
time.sleep(2)
# ページ全体のHTMLテキストを取得
html_text = driver.page_source
soup = BeautifulSoup(html_text, "html.parser")

a_list = soup.find_all('a')

for a in a_list:
    href = re.findall(r'\d+', a['href'])[0]
    if len(href) == 12:
        race_id_list.append(href)

if len(race_id_list) != 0:
    #重複削除
    race_id_list = list(set(race_id_list))
    race_id_list.sort()


In [12]:
race_id_list

['202505010101',
 '202505010102',
 '202505010103',
 '202505010104',
 '202505010105',
 '202505010106',
 '202505010107',
 '202505010108',
 '202505010109',
 '202505010110',
 '202505010111',
 '202505010112',
 '202508010101',
 '202508010102',
 '202508010103',
 '202508010104',
 '202508010105',
 '202508010106',
 '202508010107',
 '202508010108',
 '202508010109',
 '202508010110',
 '202508010111',
 '202508010112',
 '202510010301',
 '202510010302',
 '202510010303',
 '202510010304',
 '202510010305',
 '202510010306',
 '202510010307',
 '202510010308',
 '202510010309',
 '202510010310',
 '202510010311',
 '202510010312']